In [ ]:
import pandas as pd

data = pd.read_csv('c:/Users/dani/PycharmProjects/WOW/CSV/TCUSTCOUPONITEMSSPENT.csv', index_col=None)
data = data.loc[(data.ACCOUNT_CODE == 10869649) & (data.DEAL_CODE == 20) & (data.COUPON_CODE == 2158)]
data.to_csv('User10869649.csv')
print(data)

In [ ]:
data = pd.read_csv('User10869649.csv')
d = "2013-W26"
import datetime
data['datestr'] = data[['YEARIND', 'WEEKIND']].apply(lambda x: '20{}-W{}-0'.format(x.YEARIND, x.WEEKIND), axis=1)
data['date'] = pd.to_datetime(data.datestr , format='%Y-W%W-%w')
print(data)

In [ ]:
import numpy as np
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)
data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
#data = data.combine_first(data2).fillna(0) #method='ffill'
data = pd.merge(left=data, right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
data['SPENT2'] = data.SPENT_x + data.SPENT_y
data.SPENT2 = data.SPENT2.fillna(0)
data = data.fillna(method='ffill')
print(data)
data.to_csv('User2.csv')

In [56]:
import pandas as pd
data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)

def f(group):
   test = pd.merge(left=group.reset_index(), right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
   test['SPENT2'] = test.SPENT_x + test.SPENT_y
   test.SPENT2 = test.SPENT2.fillna(0)
   test = test.fillna(method='ffill')
   #print(test)
   return group.SPENT.min()


grouped = data.groupby(['DEAL_CODE', 'ACCOUNT_CODE', 'COUPON_CODE']).apply(f)
print(grouped)
grouped[20, 200, 100]

DEAL_CODE  ACCOUNT_CODE  COUPON_CODE
20         200           100            90
           10869649      2158           90
dtype: int64


90

In [278]:
import numpy as np
import pandas as pd

def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series.iloc[i+slen] - series.iloc[i]) / slen
    return sum / slen

def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen) 
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series.compute()[slen*j:slen*j+slen])/float(slen))
        #season_averages.append(1)
        #print(season_averages[0])
    #print('n_seasons: ', n_seasons, ' season_averages len: ', len(season_averages))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            #print(series.compute().iloc[0])
            sum_of_vals_over_avg += series.compute().iloc[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals

def triple_exponential_smoothing(series, slen=4, alpha=0.716, beta=0.029, gamma=0.993, n_preds=1):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series.compute().iloc[0]
            trend = initial_trend(series.compute(), slen)
            result.append(series.compute().iloc[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series.compute().iloc[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

In [190]:
import pandas as pd

data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
#data = pd.read_csv('c:/Users/dani/PycharmProjects/WOW/CSV/TCUSTCOUPONITEMSSPENT.csv', index_col=None).reset_index(drop=True)
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)

def f(group):
   df = pd.merge(left=group.reset_index(), right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
   df['SPENT2'] = df.SPENT_x + df.SPENT_y
   df.SPENT2 = df.SPENT2.fillna(0)
   df['DATE'] = 100*df.YEARIND + df.WEEKIND
   df.sort_values(['DATE'], inplace=True)
   df = df.fillna(method='ffill')
   df = df.dropna()
   df = df.reset_index()
   hws = triple_exponential_smoothing(df.SPENT2)
   #df.to_csv('User.csv')
   return hws[-1] #group.SPENT.min()

grouped = data.groupby(['DEAL_CODE', 'ACCOUNT_CODE', 'COUPON_CODE']).apply(f)
#grouped.to_csv('c:/Users/dani/PycharmProjects/WOW/CSV/TCUSTCOUPONITEMSSPENT_predicted.csv')

4
4
4


In [289]:
import dask.dataframe as dd
import pandas as pd

data2 = dd.read_csv('User1.csv')
data2['SPENT'] = 0
data = dd.read_csv('TCUST.csv')
#data = dd.read_csv('User10869649.csv')
print(data.head())

def f(group):
   df = dd.merge(left=group.reset_index(), right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
   df['SPENT2'] = df.SPENT_x + df.SPENT_y
   df.SPENT2 = df.SPENT2.fillna(0)
   df['DATE'] = 100*df.YEARIND + df.WEEKIND
   #df.sort_values(['DATE'], inplace=True) # Dask dataframe doens't support sort_values()
   df = df.fillna(method='ffill')
   df = df.dropna()
   df = df.set_index('DATE')
   #print(df.SPENT2.compute().size)
   hws = triple_exponential_smoothing(df.SPENT2)
   return hws[-1]

#group_result = pd.DataFrame(columns=['DEAL_CODE', 'ACCOUNT_CODE', 'COUPON_CODE', 'VALUE'])
series_result = pd.Series()
grouped = data.groupby(['DEAL_CODE', 'ACCOUNT_CODE', 'COUPON_CODE']).apply(f, meta=series_result)
#type(grouped)
#len(grouped)
#print(grouped.compute())
#grouped.visualize()
grouped.to_csv('testgroup')
#grouped
#grouped.where(grouped>200)
#grouped[20, 200, 100] not working
#for group in grouped:
#    print (grouped.get_group(group))
#grouped.values.compute()

   DEAL_CODE  ACCOUNT_CODE  COUPON_CODE  SPENT  WEEKIND  YEARIND
0         20          3597         2123    510       39       16
1         20          3597         2123    510       44       16
2         20          3597         2159    330       52       16
3         20          3597         2140    600       13       17
4         20          3597         2140    636       20       17


KeyboardInterrupt: 